In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageOps
import torch
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn
from torch import nn, optim
from torchvision import transforms
from torch import autograd
import torch.nn.functional as F
import os, os.path
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from torchinfo import summary
from torchsummary import summary
from torchvision import models
from sklearn.preprocessing import OneHotEncoder
import re
import itertools
import functools
import operator
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:
dataset = 1 # LH agg = 1, LH plan = 2, LH gr = 3, LH col = 4, TH gr = 5, KA plan = 6, all data combined = 7
correlation = 2 # Holzer and Sommerfeld = 1, Mola = 2, No correlation, pure Cd prediction = 3
custom_loss = 1 # 1 = True, 2 = False. Custom loss turned off when correlation = 3
pretrained = 1 # 1 = True, 2 = False. Turned off when correlation = 3
rstate = 1 #random 5 kfold splits. Any integer; the same integer will always yield the same splits

In [3]:
## Set directory for images and data
dir_LH = '/Users/Crazz/Research Codes/LH_dataset.csv'
# dir_TH = 
# dir_KA =
dir_LHim = '/Users/Crazz/Research Codes/256res_LH'
# dir_KAim =
# dir_KAim =

## Set directory for desired pre-trained model to load in
dir_premodel = '/Users/Crazz/Research Codes/WORKING MODELS/Mola models/aggmola5model.pth'
## Set directory and name for saving model. Will overwrite a file of the same name
dir_model = '/Users/Crazz/Research Codes/WORKING MODELS/LHagg_HS_model_k' #split num 0-4 and file ext .pth automatically added
## Set directory and name for saving prediction results from each split of a model over the whole dataset
dir_savdat = '/Users/Crazz/Research Codes/WORKING MODELS/LHagg_HS_outs.csv'

In [ ]:
## LOAD ALL DATASETS
LH_newdf = pd.read_csv(dir_LH)
ids = []
for i in range(len(LH_newdf)):
    picnum = str(LH_newdf['Pic Number'][i])
    rollnum = str(LH_newdf['Roll Number'][i])
    if len(picnum) == 1:
        picnum = '0' + picnum
    if len(rollnum) == 1:
        rollnum = '0' + rollnum
    ids.append(rollnum + '_' + picnum + '.png')
LH_newdf['id'] = ids
LH_newdf = LH_newdf[LH_newdf['Cd']<13]
imgs = {}
path = dir_LHim
valid_images = [".jpg",".gif",".png",".tga"]
for f in os.listdir(path):
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    temp = Image.open(os.path.join(path,f))
    imgs[f] = temp.copy()
    temp.close()

img_df = pd.DataFrame(imgs.items(), columns=['id', 'images'])
LH_newdf = LH_newdf.merge(img_df, how='inner', on='id')
LH_newdf['t_Re'] = np.log(LH_newdf['Re'])
LH_newdf['t_Cd'] = np.log(LH_newdf['Cd'])
LH_agg = LH_newdf[LH_newdf['aggregate']==1].reset_index(drop=True)
LH_agg['tmp_idx'] = range(len(LH_agg))
LH_plan = LH_newdf[LH_newdf['planar_crystal']==1].reset_index(drop=True)
LH_plan['tmp_idx'] = range(len(LH_plan))
LH_gr = LH_newdf[LH_newdf['graupel']==1].reset_index(drop=True)
LH_gr['tmp_idx'] = range(len(LH_gr))
LH_col = LH_newdf[LH_newdf['columnar_crystal']==1].reset_index(drop=True)
LH_col['tmp_idx'] = range(len(LH_col))
LH_comb = LH_newdf[LH_newdf['combo']==1].reset_index(drop=True)             #NONE
LH_comb['tmp_idx'] = range(len(LH_comb))

Theis_df = pd.read_csv(dir_TH)
imgs = {}
path = dir_THim
valid_images = [".jpg",".gif",".png",".tga"]
for f in os.listdir(path):
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    temp = Image.open(os.path.join(path,f))
    imgs[f] = temp.copy()
    temp.close()

img_df = pd.DataFrame(imgs.items(), columns=['id', 'images'])
Theis_df = Theis_df.merge(img_df, how='inner', on='id')
Theis_df = Theis_df.rename(columns={'CHA':'area','Dmax':'diam','Mass':'mass','Vel':'vel'})
Theis_df['t_Re'] = np.log(Theis_df['Re'])
Theis_df['t_Cd'] = np.log(Theis_df['Cd'])
Theis_df['tmp_idx'] = np.arange(0,len(Theis_df),1) 

KA_df = pd.read_csv(dir_KA)
imgs = {}
path = dir_KAim
valid_images = [".jpg",".gif",".png",".tga"]
for f in os.listdir(path):
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    temp = Image.open(os.path.join(path,f))
    imgs[f] = temp.copy()
    temp.close()

img_df = pd.DataFrame(imgs.items(), columns=['id', 'images'])
KA_df = KA_df.merge(img_df, how='inner', on='id')
KA_df['t_Re'] = np.log(KA_df['Re'])
KA_df['t_Cd'] = np.log(KA_df['Cd'])
KA_df['tmp_idx'] = range(len(KA_df))

alldat = pd.concat([KA_df,LH_newdf,Theis_df]).reset_index(drop=True)
alldat['t_Re'] = np.log(alldat['Re'])
alldat['t_Cd'] = np.log(alldat['Cd'])
alldat['tmp_idx'] = range(len(alldat))

In [4]:
import shutil
def save_ckp(state, is_best, checkpoint_dir, best_model_dir):
    f_path = checkpoint_dir / 'checkpoint.pt'
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_dir / 'best_model.pt'
        shutil.copyfile(f_path, best_fpath)
def load_ckp(checkpoint_fpath, model, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer, checkpoint['epoch']
                                                                                                                                                                                                                               

class regloader(torch.utils.data.Dataset):
    def __init__(self, X):
        self.X = X['images']
        self.label = X['tmp_idx']
        self.A_char = X['area']
        self.Re = X['t_Re']
        self.Cd = X['t_Cd']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = (self.X[idx])
        label = torch.tensor(self.label[idx])
        A_char = torch.tensor(self.A_char[idx], dtype=torch.float32)
        Re = torch.tensor(self.Re[idx], dtype=torch.float32)
        Cd = torch.tensor(self.Cd[idx], dtype=torch.float32)
        
        transform = transforms.Compose([
            transforms.RandomInvert(0.5),
            transforms.RandomHorizontalFlip(0.5),
            transforms.ToTensor(),
            transforms.Normalize((0.5), (0.5))]) # might need to change
        # seperate ys
        return label,transform(X),A_char,Re,Cd
    
class KGCNN1(nn.Module):
    def __init__(self):
        super(KGCNN1, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 5),#, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 48, 3),# stride=1, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(48, 64, 3),# stride=1, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 80, 3),# stride=1, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
        )
        self.geom = nn.Sequential(
            nn.Linear(80 * 14 * 14, 50), # 48*12*12 if size is 256, *4*4 128
            nn.ReLU(True),
            nn.Dropout(0.15),
            nn.Linear(50, 7), # 7 geoms return
            nn.Softplus(),
        )
    def forward(self, image):
        top_out = self.conv(image)
        middle = top_out.view(top_out.size(0), -1)
        geoms_out = self.geom(middle)
        #print(image.size())
        #print(middle1.size())
        #print(data.size())
        return geoms_out
    
class KGCNN2(nn.Module):
    def __init__(self):
        super(KGCNN2, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 5),#, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 48, 3),# stride=1, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(48, 64, 3),# stride=1, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 80, 3),# stride=1, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
        )
        self.geom = nn.Sequential(
            nn.Linear(80 * 14 * 14, 50), # 48*12*12 if size is 256, *4*4 128
            nn.ReLU(True),
            nn.Dropout(0.15),
            nn.Linear(50, 8), # 8 geoms return
            nn.Softplus(),
        )
    def forward(self, image):
        top_out = self.conv(image)
        middle = top_out.view(top_out.size(0), -1)
        geoms_out = self.geom(middle)
        #print(image.size())
        #print(middle1.size())
        #print(data.size())
        return geoms_out
class KGCNN3(nn.Module):
    def __init__(self):
        super(KGCNN3, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 5),#, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 48, 3),# stride=1, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(48, 64, 3),# stride=1, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 80, 3),# stride=1, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
        )
        self.geom = nn.Sequential(
            nn.Linear(80 * 14 * 14+1, 50), # 48*12*12 if size is 256, *4*4 128
            nn.ReLU(True),
            nn.Dropout(0.15),
            nn.Linear(50, 1), # 7 geoms return
        )
    def forward(self, image,Re):
        top_out = self.conv(image)
        middle = top_out.view(top_out.size(0), -1)
        middle_cd = torch.cat((middle, Re),dim=1)
        Cd_out = self.geom(middle_cd)
        
        #print(image.size())
        #print(middle1.size())
        #print(data.size())
        return Cd_out
    
# Custom physics loss
def physics_loss(sar, sph, length_sph, cross_sph, A_crat, A_lrat):
    tiny = 0.000000001
    loss_c = (sph/(4*sar) - A_crat*cross_sph)**2
    loss_l = (sph/(4*sar) - length_sph*(1/(2*sar) - A_lrat))**2
    loss_c = torch.mean(loss_c)
    loss_l = torch.mean(loss_l)
    # penalize geom outputs if outside range
    if torch.max(porosity)>=1:
        loss_por = 100*(porosity-0.9)**2
    else:
        loss_por = torch.tensor(0.000000001, requires_grad=True)
    if torch.max(sar) >= 0.5:
        loss_sar = 100*(sar-0.4)**2
    else:
        loss_sar = torch.tensor(0.000000001, requires_grad=True)
    if torch.max(sph) >= 1:
        loss_sph = 100*(sph-0.9)**2
    else:
        loss_sph = torch.tensor(0.000000001, requires_grad=True)
    loss_pg = torch.mean(loss_c + loss_l + loss_por + loss_sar + loss_sph)
    return loss_c, loss_l, loss_pg

## Holzer and Sommerfeld correlation
def HnS(Re,sph,lsph,csph):
    Cd_HS = 8/(Re*torch.sqrt(lsph)) + 16/(Re*torch.sqrt(sph)) + 3/(torch.sqrt(Re)*sph**(3/4)) + (1/csph)*0.4210**(0.4*((-torch.log(sph))**0.2))
    return Cd_HS
## Mola et al. correlation
def Mola(Re,por,sph,lsph,csph,Df):
    a1 = 0.10861
    a2 = 0.28273
    a3 = 0.21479
    a4 = 0.65317
    a5 = 1.49629
    b1 = 8.48137
    b2 = 5.07235
    b3 = 0.44850
    b4 = -0.71
    b5 = -1.89037
    c1 = 0.66886
    c2 = 0.14686
    c3 = -0.84224
    c4 = 0.02821
    c5 = -3.54344
    lam1 = 8*(a1*por+a2)*(a3*sph+a4)*Df**a5
    lam2 = 16*(b1*por+b2)*(b3*sph+b4)*Df**b5
    lam3 = c1*(por**((c2*(1-por)/por)**c3))*(Re**c4)*sph**c5
    Cd_M = lam1/(Re*torch.sqrt(csph)) + lam2/(Re*torch.sqrt(sph)) + lam3*(3/(torch.sqrt(Re)*sph**(3/4)) + (1/csph)*0.4210**(0.4*((-torch.log(sph))**0.2)))
    return Cd_M

if dataset == 1: # LH agg
    k_folds = 5
    batch_size = 2
    kf = KFold(n_splits=k_folds, shuffle=True,random_state=rstate)
    lr = 1e-6
    num_epochs = 750
    fulldata = regloader(LH_agg)
    cur_df = LH_agg
if dataset == 2: # LH plan
    k_folds = 5
    batch_size = 2
    kf = KFold(n_splits=k_folds, shuffle=True,random_state=rstate)
    lr = 1e-6
    num_epochs = 750
    fulldata = regloader(LH_plan)
    cur_df = LH_plan
if dataset == 3: # LH graupel
    k_folds = 5
    batch_size = 6
    kf = KFold(n_splits=k_folds, shuffle=True,random_state=rstate)
    lr = 1e-5
    num_epochs = 750
    fulldata = regloader(LH_gr)
    cur_df = LH_gr
if dataset == 4: # LH col
    k_folds = 5
    batch_size = 2
    kf = KFold(n_splits=k_folds, shuffle=True,random_state=rstate)
    lr = 1e-6
    num_epochs = 750
    fulldata = regloader(LH_col)
    cur_df = LH_col
if dataset == 5: # TH graupel
    k_folds = 5
    batch_size = 2
    kf = KFold(n_splits=k_folds, shuffle=True,random_state=rstate)
    lr = 1e-6
    num_epochs = 750
    fulldata = regloader(Theis_df)
    cur_df = Theis_df
if dataset == 6: # KA planar
    k_folds = 5
    batch_size = 2
    kf = KFold(n_splits=k_folds, shuffle=True,random_state=rstate)
    lr = 1e-6
    num_epochs = 750
    fulldata = regloader(KA_df)
    cur_df = KA_df
if dataset == 7: # all data combined
    k_folds = 5
    batch_size = 10
    kf = KFold(n_splits=k_folds, shuffle=True,random_state=rstate)
    lr = 1e-5
    num_epochs = 750
    fulldata = regloader(alldata)
    cur_df = alldata

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if correlation == 1
    model = KGCNN1.to(device)
if correlation == 2
    model = KGCNN2.to(device)
if correlation == 3
    model = KGCNN3.to(device)


criterion = nn.MSELoss() #MSELoss()  # Mean squared error loss #L1Loss()


# Training loop
if correlation == 1:
    train_Cd_curves = []
    train_pg_curves = []

    test_Cd_curves = []
    test_pg_curves = []

    goodplanmodel_epoch = np.array([0,0,0,0,0])

    best_test_acc = np.array([100,100,100,100,100])
    for fold, (train_ids, test_ids) in enumerate(kf.split(fulldata)):
        print(f'FOLD {fold}')
        print('--------------------------------')
        train_loader = DataLoader(
            dataset=fulldata,
            batch_size=batch_size,
            sampler=torch.utils.data.SubsetRandomSampler(train_ids),
        )
        test_loader = DataLoader(
            dataset=fulldata,
            batch_size=batch_size,
            sampler=torch.utils.data.SubsetRandomSampler(test_ids),
        )
        model = KGCNN1.to(device)
        non_frozen_parameters = [p for p in model.parameters() if p.requires_grad]
        optimizer = torch.optim.Adam(non_frozen_parameters, lr=lr)  # Choose an optimizer
        if pretrained == 1:
            model, optimizer, start_epoch = load_ckp(dir_premodel, model, optimizer)

        best_test = best_test_acc[fold]
        train_llist_Cd = []
        train_llist_pg = []

        test_llist_Cd = []
        test_llist_pg = []

        for epoch in range(num_epochs):
            model.train()  # Set the model in training mode

            train_Cd_loss = 0.0
            train_pg_loss = 0.0

            for id,images,A_char,Re,Cd in train_loader:

                id = id.to(device)
                images = Variable(images, requires_grad=True)
                A_char = Variable(A_char, requires_grad=True)
                Re = Variable(Re, requires_grad=True)
                Cd = Variable(Cd, requires_grad=True)

                A_char = A_char.view(A_char.size(0),1)
                Re = Re.view(Re.size(0),1)
                Cd = Cd.view(Cd.size(0),1)

                images = images.to(device)
                A_char = A_char.to(device)
                Re = Re.to(device)
                Cd = Cd.to(device)

                #with autograd.detect_anomaly():
                # Forward pass
                geoms_out = model(images)
                sph = geoms_out[:,0]
                cross_sph = geoms_out[:,1]
                length_sph = geoms_out[:,2]
                porosity = geoms_out[:,3]
                sar = geoms_out[:,4]
                A_crat = geoms_out[:,5]
                A_lrat = geoms_out[:,6]

                sph = sph.view(Re.size(0),1)
                cross_sph = cross_sph.view(Re.size(0),1)
                length_sph = length_sph.view(Re.size(0),1)
                porosity = porosity.view(Re.size(0),1)
                sar = sar.view(Re.size(0),1)
                A_crat = A_crat.view(A_crat.size(0),1)
                A_lrat = A_lrat.view(A_lrat.size(0),1)

                loss_c,loss_l,loss_pg = physics_loss(sar*0.5,sph,length_sph*2,cross_sph*3.5,A_crat*1.5,A_lrat*1.5)

                Cd_act = HnS(torch.exp(Re),sph,length_sph*2,cross_sph*3.5)#*(torch.abs(1 - porosity))**0.17)*torch.exp(Re)**-0.13)
                Cd_logout = torch.log(Cd_act)

                # TOTAL loss w/o phys Cd
                # sum physics guided and regular losses from Cd and geom parameters
                if torch.max(sph) >= 1:
                    loss_Cd = 2*torch.abs(torch.max(sph)-0.5)**2
                else:
                    loss_Cd = criterion(Cd_logout,Cd)
                loss_Cd = loss_Cd.to(device)
                loss_tot = loss_Cd + loss_pg

                # Backward pass and optimization
                if custom_loss == 1:
                    loss_tot.backward()
                if custom_loss == 2:
                    loss_Cd.backward()
                #nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                clip_value = 10
                torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
                optimizer.step()
                optimizer.zero_grad()

                #####
                train_Cd_loss += loss_Cd * id.size(0)
                train_pg_loss += loss_pg * id.size(0)

            train_Cd_loss /= len(train_loader.dataset)
            train_pg_loss /= len(train_loader.dataset)

            train_llist_pg.append(train_pg_loss.item())

            # Evaluation
            model.eval()  # Set the model in evaluation mode

            test_Cd_loss = 0.0
            test_pg_loss = 0.0

            with torch.no_grad():
                for id,images,A_char,Re,Cd in test_loader:

                    id = id.to(device)
                    images = Variable(images, requires_grad=True)
                    A_char = Variable(A_char, requires_grad=True)
                    Re = Variable(Re, requires_grad=True)
                    Cd = Variable(Cd, requires_grad=True)

                    A_char = A_char.view(A_char.size(0),1)
                    Re = Re.view(Re.size(0),1)
                    Cd = Cd.view(Cd.size(0),1)

                    images = images.to(device)
                    A_char = A_char.to(device)
                    Re = Re.to(device)
                    Cd = Cd.to(device)

                    #with autograd.detect_anomaly():
                    # Forward pass
                    geoms_out = model(images)
                    sph = geoms_out[:,0]
                    cross_sph = geoms_out[:,1]
                    length_sph = geoms_out[:,2]
                    porosity = geoms_out[:,3]
                    sar = geoms_out[:,4]
                    A_crat = geoms_out[:,5]
                    A_lrat = geoms_out[:,6]

                    sph = sph.view(Re.size(0),1)
                    cross_sph = cross_sph.view(Re.size(0),1)
                    length_sph = length_sph.view(Re.size(0),1)
                    porosity = porosity.view(Re.size(0),1)
                    sar = sar.view(Re.size(0),1)
                    A_crat = A_crat.view(A_crat.size(0),1)
                    A_lrat = A_lrat.view(A_lrat.size(0),1)

                    loss_c,loss_l,loss_pg = physics_loss(sar*0.5,sph,length_sph*2,cross_sph*3.5,A_crat*1.5,A_lrat*1.5)

                    Cd_act = HnS(torch.exp(Re),sph,length_sph*2,cross_sph*3.5)#*(torch.abs(1 - porosity))**0.17)*torch.exp(Re)**-0.13)
                    Cd_logout = torch.log(Cd_act)

                    # TOTAL loss w/o phys Cd
                    # sum physics guided and regular losses from Cd and geom parameters
                    loss_Cd = criterion(Cd_logout,Cd)

                    loss_tot = loss_Cd + loss_pg
                    # TOTAL loss w/o phys Cd
                    # sum physics guided and regular losses from Cd and geom parameters
                    test_Cd_loss += loss_Cd * id.size(0)
                    test_pg_loss += loss_pg * id.size(0)

            test_Cd_loss /= len(test_loader.dataset)
            test_pg_loss /= len(test_loader.dataset)

            test_llist_Cd.append(test_Cd_loss.item())
            test_llist_pg.append(test_pg_loss.item())

            if test_Cd_loss < best_test:
                torch.save(model.state_dict(), dir_model+str(fold)+'.pth')
                goodplanmodel_epoch[fold] = epoch
                best_test = test_Cd_loss

            print(f"Epoch {epoch+1}/{num_epochs} - Train Cd Loss: {train_Cd_loss:.6f} - Test Loss: {test_Cd_loss:.6f}")
            print(f"Epoch {epoch+1}/{num_epochs} - Train pg Loss: {train_pg_loss:.6f} - Test Loss: {test_pg_loss:.6f}")
            print('\n')
            #print(Cd_out)
        train_Cd_curves.append(train_llist_Cd)
        test_Cd_curves.append(test_llist_Cd)
        train_pg_curves.append(train_llist_pg)
        test_pg_curves.append(test_llist_pg)

if correlation == 2:
    train_Cd_curves = []
    train_pg_curves = []

    test_Cd_curves = []
    test_pg_curves = []

    best_test_acc = np.array([100,100,100,100,100])

    # Training loop

    for fold, (train_ids, test_ids) in enumerate(kf.split(fulldata)):
        print(f'FOLD {fold}')
        print('--------------------------------')
        train_loader = DataLoader(
            dataset=fulldata,
            batch_size=batch_size,
            sampler=torch.utils.data.SubsetRandomSampler(train_ids),
        )
        test_loader = DataLoader(
            dataset=fulldata,
            batch_size=batch_size,
            sampler=torch.utils.data.SubsetRandomSampler(test_ids),
        )
        model = KGCNN2.to(device)
        non_frozen_parameters = [p for p in model.parameters() if p.requires_grad]
        optimizer = torch.optim.Adam(non_frozen_parameters, lr=lr)  # Choose an optimizer
        
        if pretrained == 1:
            model, optimizer, start_epoch = load_ckp(dir_premodel, model, optimizer)
    
        best_test = best_test_acc[fold]

        train_llist_Cd = []
        train_llist_pg = []
        test_llist_Cd = []
        test_llist_pg = []

        for epoch in range(num_epochs):
            model.train()  # Set the model in training mode

            train_Cd_loss = 0.0
            train_Df_loss = 0.0
            train_pg_loss = 0.0

            for id,images,A_char,Re,Cd,Df_dat in train_loader:

                id = id.to(device)
                images = Variable(images, requires_grad=True)
                A_char = Variable(A_char, requires_grad=True)
                Re = Variable(Re, requires_grad=True)
                Cd = Variable(Cd, requires_grad=True)
                Df_dat = Variable(Df_dat, requires_grad=True)

                A_char = A_char.view(A_char.size(0),1)
                Re = Re.view(Re.size(0),1)
                Cd = Cd.view(Cd.size(0),1)
                Df_dat = Df_dat.view(Df_dat.size(0),1)

                images = images.to(device)
                A_char = A_char.to(device)
                Re = Re.to(device)
                Cd = Cd.to(device)
                Df_dat = Df_dat.to(device)

                #with autograd.detect_anomaly():
                # Forward pass
                geoms_out = model(images)
                sph = geoms_out[:,0]
                cross_sph = geoms_out[:,1]
                length_sph = geoms_out[:,2]
                porosity = geoms_out[:,3]
                sar = geoms_out[:,4]
                A_crat = geoms_out[:,5]
                A_lrat = geoms_out[:,6]
                Df = geoms_out[:,7]

                sph = sph.view(Re.size(0),1)
                cross_sph = cross_sph.view(Re.size(0),1)
                length_sph = length_sph.view(Re.size(0),1)
                porosity = porosity.view(Re.size(0),1)
                sar = sar.view(Re.size(0),1)
                A_crat = A_crat.view(A_crat.size(0),1)
                A_lrat = A_lrat.view(A_lrat.size(0),1)
                Df = Df.view(Df.size(0),1)

                loss_c,loss_l,loss_pg = physics_loss(sar*0.5,sph,length_sph*2,cross_sph*3.5,A_crat*1.5,A_lrat*1.5)

                Cd_act = Mola(torch.exp(Re),porosity,sph,length_sph*2,cross_sph*3.5,Df*3*1.3)
                Cd_logout = torch.log(Cd_act)

                # TOTAL loss w/o phys Cd
                # sum physics guided and regular losses from Cd and geom parameters
                if torch.max(sph) >= 1 or torch.max(porosity) >= 1:
                    loss_Cd = torch.tensor(0.2,requires_grad=True)
                elif torch.min(Cd_act) <= 0.0:
                    loss_Cd = torch.abs(torch.min(Cd_act)*50)
                else:
                    loss_Cd = criterion(Cd_logout,Cd)

                if torch.max(Df*3*1.3) >= 3:
                    loss_Df = torch.abs(torch.max(Df*3*1.3)-2.9)**2
                elif torch.min(Df*3*1.3) <= 1.4:
                    loss_Df = torch.abs(torch.min(Df*3*1.3)-1.5)**2
                else:
                    loss_Df = criterion(Df*3*1.3,Df_dat*3*1.3)/8

                loss_Df = loss_Df.to(device)
                loss_Cd = loss_Cd.to(device)
                loss_tot = loss_Cd + loss_Df + loss_pg

                # Backward pass and optimization
                if custom_loss == 1:
                    loss_tot.backward()
                if custom_loss == 2:
                    (loss_Cd+loss_Df).backward()
                
                #nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                clip_value = 10
                torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
                optimizer.step()
                optimizer.zero_grad()

                #####
                train_Cd_loss += loss_Cd * id.size(0)
                train_Df_loss += loss_Df* id.size(0)
                train_pg_loss += loss_pg * id.size(0)

            train_Cd_loss /= len(train_loader.dataset)
            train_Df_loss /= len(train_loader.dataset)
            train_pg_loss /= len(train_loader.dataset)

            train_llist_pg.append(train_pg_loss.item())

            # Evaluation
            model.eval()  # Set the model in evaluation mode

            test_Cd_loss = 0.0
            test_Df_loss = 0.0
            test_pg_loss = 0.0

            with torch.no_grad():
                for id,images,A_char,Re,Cd,Df_dat in test_loader:

                    id = id.to(device)
                    images = Variable(images, requires_grad=True)
                    A_char = Variable(A_char, requires_grad=True)
                    Re = Variable(Re, requires_grad=True)
                    Cd = Variable(Cd, requires_grad=True)
                    Df_dat = Variable(Df_dat, requires_grad=True)

                    A_char = A_char.view(A_char.size(0),1)
                    Re = Re.view(Re.size(0),1)
                    Cd = Cd.view(Cd.size(0),1)
                    Df_dat = Df_dat.view(Df_dat.size(0),1)

                    images = images.to(device)
                    A_char = A_char.to(device)
                    Re = Re.to(device)
                    Cd = Cd.to(device)
                    Df_dat = Df_dat.to(device)

                    #with autograd.detect_anomaly():
                    # Forward pass
                    geoms_out = model(images)
                    sph = geoms_out[:,0]
                    cross_sph = geoms_out[:,1]
                    length_sph = geoms_out[:,2]
                    porosity = geoms_out[:,3]
                    sar = geoms_out[:,4]
                    A_crat = geoms_out[:,5]
                    A_lrat = geoms_out[:,6]
                    Df = geoms_out[:,7]

                    sph = sph.view(Re.size(0),1)
                    cross_sph = cross_sph.view(Re.size(0),1)
                    length_sph = length_sph.view(Re.size(0),1)
                    porosity = porosity.view(Re.size(0),1)
                    sar = sar.view(Re.size(0),1)
                    A_crat = A_crat.view(A_crat.size(0),1)
                    A_lrat = A_lrat.view(A_lrat.size(0),1)
                    Df = Df.view(Df.size(0),1)

                    loss_c,loss_l,loss_pg = physics_loss(sar*0.5,sph,length_sph*2,cross_sph*3.5,A_crat*1.5,A_lrat*1.5)

                    Cd_act = Mola(torch.exp(Re),porosity,sph,length_sph*2,cross_sph*3.5,Df*3*1.3)
                    Cd_logout = torch.log(Cd_act)

                    # TOTAL loss w/o phys Cd
                    # sum physics guided and regular losses from Cd and geom parameters
                    loss_Cd = criterion(Cd_logout,Cd)
                    loss_Df = criterion(Df*3*1.3,Df_dat*3*1.3)/8
                    loss_tot = loss_Cd + loss_Df + loss_pg
                    # TOTAL loss w/o phys Cd
                    # sum physics guided and regular losses from Cd and geom parameters
                    test_Cd_loss += loss_Cd * id.size(0)
                    test_Df_loss += loss_Df * id.size(0)
                    test_pg_loss += loss_pg * id.size(0)

            test_Cd_loss /= len(test_loader.dataset)
            test_Df_loss /= len(test_loader.dataset)
            test_pg_loss /= len(test_loader.dataset)

            test_llist_Cd.append(test_Cd_loss.item())
            test_llist_pg.append(test_pg_loss.item())

            if test_Cd_loss < best_test:
                torch.save(model.state_dict(), dir_model+str(fold)+'.pth')
                goodplanmodel_epoch = epoch
                best_test = test_Cd_loss

            print(f"Epoch {epoch+1}/{num_epochs} - Train Cd Loss: {train_Cd_loss:.6f} - Test Loss: {test_Cd_loss:.6f}")
            print(f"Epoch {epoch+1}/{num_epochs} - Train Df Loss: {train_Df_loss:.6f} - Test Loss: {test_Df_loss:.6f}")
            print(f"Epoch {epoch+1}/{num_epochs} - Train pg Loss: {train_pg_loss:.6f} - Test Loss: {test_pg_loss:.6f}")
            print('\n')
            #print(Cd_out)
        train_Cd_curves.append(train_llist_Cd)
        test_Cd_curves.append(test_llist_Cd)
        train_pg_curves.append(train_llist_pg)
        test_pg_curves.append(test_llist_pg)

if correlation == 3:
    train_llist_Cd = []
    test_llist_Cd = []
    goodplanmodel_epoch = np.array([0,0,0,0,0])

    best_test_acc = np.array([100,100,100,100,100])

    # Training loop

    for fold, (train_ids, test_ids) in enumerate(kf.split(fulldata)):
        print(f'FOLD {fold}')
        print('--------------------------------')
        train_loader = DataLoader(
            dataset=fulldata,
            batch_size=batch_size,
            sampler=torch.utils.data.SubsetRandomSampler(train_ids),
        )
        test_loader = DataLoader(
            dataset=fulldata,
            batch_size=batch_size,
            sampler=torch.utils.data.SubsetRandomSampler(test_ids),
        )
        model = KGCNN3.to(device)
        non_frozen_parameters = [p for p in model.parameters() if p.requires_grad]
        optimizer = torch.optim.Adam(non_frozen_parameters, lr=lr)  # Choose an optimizer
        best_test = best_test_acc[fold]

        for epoch in range(num_epochs):
            model.train()  # Set the model in training mode

            train_Cd_loss = 0.0

            for id,images,A_char,Re,Cd in train_loader:

                id = id.to(device)
                images = Variable(images, requires_grad=True)
                A_char = Variable(A_char, requires_grad=True)
                Re = Variable(Re, requires_grad=True)
                Cd = Variable(Cd, requires_grad=True)

                A_char = A_char.view(A_char.size(0),1)
                Re = Re.view(Re.size(0),1)
                Cd = Cd.view(Cd.size(0),1)

                images = images.to(device)
                A_char = A_char.to(device)
                Re = Re.to(device)
                Cd = Cd.to(device)

                #with autograd.detect_anomaly():
                # Forward pass
                cd_out = model(images,Re)

                loss_Cd = criterion(cd_out,Cd)

                # Backward pass and optimization
                loss_Cd.backward()
                #nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                clip_value = 10
                torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
                optimizer.step()
                optimizer.zero_grad()

                #####
                train_Cd_loss += loss_Cd * id.size(0)

            train_Cd_loss /= len(train_loader.dataset)

            # Evaluation
            model.eval()  # Set the model in evaluation mode

            test_Cd_loss = 0.0

            with torch.no_grad():
                for id,images,A_char,Re,Cd in test_loader:

                    id = id.to(device)
                    images = Variable(images, requires_grad=True)
                    A_char = Variable(A_char, requires_grad=True)
                    Re = Variable(Re, requires_grad=True)
                    Cd = Variable(Cd, requires_grad=True)

                    A_char = A_char.view(A_char.size(0),1)
                    Re = Re.view(Re.size(0),1)
                    Cd = Cd.view(Cd.size(0),1)

                    images = images.to(device)
                    A_char = A_char.to(device)
                    Re = Re.to(device)
                    Cd = Cd.to(device)

                    #with autograd.detect_anomaly():
                    # Forward pass
                    cd_out = model(images,Re)

                    loss_Cd = criterion(cd_out,Cd)

                    # TOTAL loss w/o phys Cd
                    # sum physics guided and regular losses from Cd and geom parameters
                    test_Cd_loss += loss_Cd * id.size(0)

            test_Cd_loss /= len(test_loader.dataset)

            test_llist_Cd.append(test_Cd_loss.item())

            if test_Cd_loss < best_test:
                torch.save(model.state_dict(), dir_model+str(fold)+'.pth')
                goodplanmodel_epoch[fold] = epoch
                best_test = test_Cd_loss

            print(f"Epoch {epoch+1}/{num_epochs} - Train Cd Loss: {train_Cd_loss:.6f} - Test Loss: {test_Cd_loss:.6f}")
            print('\n')
print('Completed training.')
print('Now evaluating performance')

if correlation == 1:
    hs_planout = pd.DataFrame(columns=['por','sar','sph','lsph','csph','Acrat','Alrat','Cdout','massout','Rhos','Rhoc','Rhol','id','Cd','Re','mass'])
    x,y,z,w,v = kf.split(fulldata)
    model = KGCNN1.to(device)
    for i in range(5):
        model.load_state_dict(torch.load(dir_model+str(i)+".pth"))
        if i == 0:
            test_loader = DataLoader(
                dataset=fulldata,
                batch_size=1,
                sampler=torch.utils.data.SubsetRandomSampler(x[1])
            )
        elif i == 1:
            test_loader = DataLoader(
            dataset=fulldata,
            batch_size=1,
            sampler=torch.utils.data.SubsetRandomSampler(y[1])
            )
        elif i == 2:
            test_loader = DataLoader(
            dataset=fulldata,
            batch_size=1,
            sampler=torch.utils.data.SubsetRandomSampler(z[1])
            )
        elif i == 3:
            test_loader = DataLoader(
            dataset=fulldata,
            batch_size=1,
            sampler=torch.utils.data.SubsetRandomSampler(w[1])
            )
        elif i == 4:
            test_loader = DataLoader(
            dataset=fulldata,
            batch_size=1,
            sampler=torch.utils.data.SubsetRandomSampler(v[1])
            )
        model.eval()  # Set the model in evaluation mode

        Cd_outs = []
        por_outs = []
        sar_outs = []
        sph_outs = []
        csph_outs = []
        lsph_outs = []
        Acrat_outs = []
        Alrat_outs = []

        Cd_ans = []

        test_ids = []
        with torch.no_grad():
            for id,images,A_char,Re,Cd in test_loader:

                id = id.to(device)
                images = Variable(images, requires_grad=True)
                A_char = Variable(A_char, requires_grad=True)
                Re = Variable(Re, requires_grad=True)
                Cd = Variable(Cd, requires_grad=True)

                A_char = A_char.view(A_char.size(0),1)
                Re = Re.view(Re.size(0),1)
                Cd = Cd.view(Cd.size(0),1)

                images = images.to(device)
                A_char = A_char.to(device)
                Re = Re.to(device)
                Cd = Cd.to(device)

                #with autograd.detect_anomaly():
                # Forward pass
                geoms_out = model(images)
                sph = geoms_out[:,0]
                cross_sph = geoms_out[:,1]
                length_sph = geoms_out[:,2]
                porosity = geoms_out[:,3]
                sar = geoms_out[:,4]
                A_crat = geoms_out[:,5]
                A_lrat = geoms_out[:,6]

                sph = sph.view(Re.size(0),1)
                cross_sph = cross_sph.view(Re.size(0),1)
                length_sph = length_sph.view(Re.size(0),1)
                porosity = porosity.view(Re.size(0),1)
                sar = sar.view(Re.size(0),1)
                A_crat = A_crat.view(A_crat.size(0),1)
                A_lrat = A_lrat.view(A_lrat.size(0),1)

                Cd_out = HnS(torch.exp(Re),sph,length_sph*2,cross_sph*3.5)#*(torch.abs(1 - porosity))**0.17)*torch.exp(Re)**-0.13)
                Cd_logout = torch.log(Cd_out)

                Cd_outs.append(Cd_out.item())
                por_outs.append(porosity.item())
                sar_outs.append(sar.item())
                sph_outs.append(sph.item())
                lsph_outs.append(length_sph.item())
                csph_outs.append(cross_sph.item())
                Acrat_outs.append(A_crat.item())
                Alrat_outs.append(A_lrat.item())

                Cd_ans.append(torch.exp(Cd).item())

                test_ids.append(id.item())
                #err += (abs(outputs - t_Cd)/t_Cd)*100
                #print(err)
                #loss =  l + loss_mean
        #outs_act = np.array(outs)
        #answers_act = np.array(answers)
        Cd_outs = np.array(Cd_outs)
        Cd_acts = np.array(Cd_outs)

        por_outs = np.array(por_outs)
        sar_outs = np.array(sar_outs)
        sph_outs = np.array(sph_outs)
        lsph_outs = np.array(lsph_outs)
        csph_outs = np.array(csph_outs)
        Acrat_outs = np.array(Acrat_outs)
        Alrat_outs = np.array(Alrat_outs)
        por_outs = por_outs
        sar_outs = sar_outs*0.5
        sph_outs = sph_outs
        l_sph_outs = lsph_outs*2
        c_sph_outs = csph_outs*3.5
        Acrat_outs = Acrat_outs*1.5
        Alrat_outs = Alrat_outs*1.5

        Cd_ans = np.array(Cd_ans)
        Cd_actans = np.array(Cd_ans)

        test_ids = np.array(test_ids)
        mass_out = Cd_acts*0.5*cur_df['rho'][test_ids]*(cur_df['vel'][test_ids]**2)*cur_df['area'][test_ids]/9.81
        Cd_errs = ((Cd_ans - Cd_outs)/abs(Cd_ans))*100
        Cd_errs_abs = (abs(Cd_ans - Cd_outs)/abs(Cd_ans))*100
        Cd_aerrs = -((Cd_actans - Cd_acts)/abs(Cd_actans))*100
        Cd_aerrs_abs = (abs(Cd_actans - Cd_acts)/abs(Cd_actans))*100
        Cd_mse = (Cd_acts - Cd_actans)**2
        Cd_rmse = np.sqrt(np.sum(Cd_mse)/len(Cd_mse))
        Cd_nrmse = (Cd_rmse/np.mean(Cd_actans))*100
        mass_mse = (mass_out - cur_df['mass'][test_ids])**2
        mass_rmse = np.sqrt(np.sum(mass_mse)/len(mass_mse))
        Vol_out_s = 4*np.pi/3*(sph_outs/(4*sar_outs) * cur_df['area'][test_ids]/np.pi)**(3/2)
        rho_s = mass_out/Vol_out_s
        Vol_out_c = 4*np.pi/3*(c_sph_outs*Acrat_outs * cur_df['area'][test_ids]/np.pi)**(3/2)
        rho_c = mass_out/Vol_out_c
        Vol_out_l = 4*np.pi/3*(l_sph_outs*(1/(2*sar_outs)-Alrat_outs) * cur_df['area'][test_ids]/np.pi)**(3/2)
        rho_l = mass_out/Vol_out_l
        predout = pd.DataFrame(por_outs,columns=['por'])
        predout['sar'] = sar_outs
        predout['sph'] = sph_outs
        predout['lsph'] = l_sph_outs
        predout['csph'] = c_sph_outs
        predout['Acrat'] = Acrat_outs
        predout['Alrat'] = Alrat_outs
        predout['Cdout'] = Cd_outs
        predout['massout'] = np.array(mass_out)
        predout['Rhos'] = np.array(rho_s)
        predout['Rhoc'] = np.array(rho_c)
        predout['Rhol'] = np.array(rho_l)
        predout['id'] = np.array(cur_df['id'][test_ids])
        predout['Cd'] = np.array(cur_df['Cd'][test_ids])
        predout['Re'] = np.array(cur_df['Re'][test_ids])
        predout['mass'] = np.array(cur_df['mass'][test_ids])
        hs_planout = pd.concat([hs_planout,predout],ignore_index=True)
        
if correlation == 2:
    hs_planout = pd.DataFrame(columns=['por','sar','sph','lsph','csph','Acrat','Alrat','Df_out','Cdout','massout','Rhos','Rhoc','Rhol','id','Cd','Re','Df','mass'])
    x,y,z,w,v = kf.split(fulldata)
    model = KGCNN2.to(device)
    for i in range(5):
        model.load_state_dict(torch.load(dir_model+str(i)+".pth"))
        if i == 0:
            test_loader = DataLoader(
                dataset=fulldata,
                batch_size=1,
                sampler=torch.utils.data.SubsetRandomSampler(x[1])
            )
        elif i == 1:
            test_loader = DataLoader(
            dataset=fulldata,
            batch_size=1,
            sampler=torch.utils.data.SubsetRandomSampler(y[1])
            )
        elif i == 2:
            test_loader = DataLoader(
            dataset=fulldata,
            batch_size=1,
            sampler=torch.utils.data.SubsetRandomSampler(z[1])
            )
        elif i == 3:
            test_loader = DataLoader(
            dataset=fulldata,
            batch_size=1,
            sampler=torch.utils.data.SubsetRandomSampler(w[1])
            )
        elif i == 4:
            test_loader = DataLoader(
            dataset=fulldata,
            batch_size=1,
            sampler=torch.utils.data.SubsetRandomSampler(v[1])
            )
        model.eval()  # Set the model in evaluation mode

        Cd_outs = []
        por_outs = []
        sar_outs = []
        sph_outs = []
        csph_outs = []
        lsph_outs = []
        Acrat_outs = []
        Alrat_outs = []
        Df_outs = []

        Cd_ans = []
        Df_ans = []

        test_ids = []
        with torch.no_grad():
            for id,images,A_char,Re,Cd,Df_dat in test_loader:

                id = id.to(device)
                images = Variable(images, requires_grad=True)
                A_char = Variable(A_char, requires_grad=True)
                Re = Variable(Re, requires_grad=True)
                Cd = Variable(Cd, requires_grad=True)
                Df_dat = Variable(Df_dat, requires_grad=True)

                A_char = A_char.view(A_char.size(0),1)
                Re = Re.view(Re.size(0),1)
                Cd = Cd.view(Cd.size(0),1)
                Df_dat = Df_dat.view(Df_dat.size(0),1)

                images = images.to(device)
                A_char = A_char.to(device)
                Re = Re.to(device)
                Cd = Cd.to(device)
                Df_dat = Df_dat.to(device)

                #with autograd.detect_anomaly():
                # Forward pass
                geoms_out = model(images)
                sph = geoms_out[:,0]
                cross_sph = geoms_out[:,1]
                length_sph = geoms_out[:,2]
                porosity = geoms_out[:,3]
                sar = geoms_out[:,4]
                A_crat = geoms_out[:,5]
                A_lrat = geoms_out[:,6]
                Df = geoms_out[:,7]

                sph = sph.view(Re.size(0),1)
                cross_sph = cross_sph.view(Re.size(0),1)
                length_sph = length_sph.view(Re.size(0),1)
                porosity = porosity.view(Re.size(0),1)
                sar = sar.view(Re.size(0),1)
                A_crat = A_crat.view(A_crat.size(0),1)
                A_lrat = A_lrat.view(A_lrat.size(0),1)
                Df = Df.view(Df.size(0),1)

                Cd_out = Mola(torch.exp(Re),porosity,sph,length_sph*2,cross_sph*3.5,Df*3*1.3)
                Cd_logout = torch.log(Cd_out)

                Cd_outs.append(Cd_out.item())
                por_outs.append(porosity.item())
                sar_outs.append(sar.item())
                sph_outs.append(sph.item())
                lsph_outs.append(length_sph.item())
                csph_outs.append(cross_sph.item())
                Acrat_outs.append(A_crat.item())
                Alrat_outs.append(A_lrat.item())
                Df_outs.append(Df.item())

                Cd_ans.append(torch.exp(Cd).item())
                Df_ans.append(Df_dat.item())

                test_ids.append(id.item())
                #err += (abs(outputs - t_Cd)/t_Cd)*100
                #print(err)
                #loss =  l + loss_mean
        #outs_act = np.array(outs)
        #answers_act = np.array(answers)
        Cd_outs = np.array(Cd_outs)
        Cd_acts = np.array(Cd_outs)

        por_outs = np.array(por_outs)
        sar_outs = np.array(sar_outs)
        sph_outs = np.array(sph_outs)
        lsph_outs = np.array(lsph_outs)
        csph_outs = np.array(csph_outs)
        Acrat_outs = np.array(Acrat_outs)
        Alrat_outs = np.array(Alrat_outs)
        Df_outs = np.array(Df_outs)
        por_outs = por_outs
        sar_outs = sar_outs*0.5
        sph_outs = sph_outs
        l_sph_outs = lsph_outs*2
        c_sph_outs = csph_outs*3.5
        Acrat_outs = Acrat_outs*1.5
        Alrat_outs = Alrat_outs*1.5
        Df_outs = Df_outs*3*1.3

        Cd_ans = np.array(Cd_ans)
        Cd_actans = np.array(Cd_ans)
        Df_ans = np.array(Df_ans)*3*1.3

        test_ids = np.array(test_ids)
        mass_out = Cd_acts*0.5*cur_df['rho'][test_ids]*(cur_df['vel'][test_ids]**2)*cur_df['area'][test_ids]/9.81
        Cd_errs = ((Cd_ans - Cd_outs)/abs(Cd_ans))*100
        Cd_errs_abs = (abs(Cd_ans - Cd_outs)/abs(Cd_ans))*100
        Cd_aerrs = -((Cd_actans - Cd_acts)/abs(Cd_actans))*100
        Cd_aerrs_abs = (abs(Cd_actans - Cd_acts)/abs(Cd_actans))*100
        Cd_mse = (Cd_acts - Cd_actans)**2
        Cd_rmse = np.sqrt(np.sum(Cd_mse)/len(Cd_mse))
        Cd_nrmse = (Cd_rmse/np.mean(Cd_actans))*100
        mass_mse = (mass_out - cur_df['mass'][test_ids])**2
        mass_rmse = np.sqrt(np.sum(mass_mse)/len(mass_mse))
        Vol_out_s = 4*np.pi/3*(sph_outs/(4*sar_outs) * cur_df['area'][test_ids]/np.pi)**(3/2)
        rho_s = mass_out/Vol_out_s
        Vol_out_c = 4*np.pi/3*(c_sph_outs*Acrat_outs * cur_df['area'][test_ids]/np.pi)**(3/2)
        rho_c = mass_out/Vol_out_c
        Vol_out_l = 4*np.pi/3*(l_sph_outs*(1/(2*sar_outs)-Alrat_outs) * cur_df['area'][test_ids]/np.pi)**(3/2)
        rho_l = mass_out/Vol_out_l
        predout = pd.DataFrame(por_outs,columns=['por'])
        predout['sar'] = sar_outs
        predout['sph'] = sph_outs
        predout['lsph'] = l_sph_outs
        predout['csph'] = c_sph_outs
        predout['Acrat'] = Acrat_outs
        predout['Alrat'] = Alrat_outs
        predout['Df_out'] = Df_outs
        predout['Cdout'] = Cd_outs
        predout['massout'] = np.array(mass_out)
        predout['Rhos'] = np.array(rho_s)
        predout['Rhoc'] = np.array(rho_c)
        predout['Rhol'] = np.array(rho_l)
        predout['id'] = np.array(cur_df['id'][test_ids])
        predout['Cd'] = np.array(cur_df['Cd'][test_ids])
        predout['Re'] = np.array(cur_df['Re'][test_ids])
        predout['Df'] = Df_ans
        predout['mass'] = np.array(cur_df['mass'][test_ids])
        hs_planout = pd.concat([hs_planout,predout],ignore_index=True)

if correlation == 3:
    hs_planout = pd.DataFrame(columns=['Cdout','massout','id','Cd','Re','mass'])
    x,y,z,w,v = kf.split(fulldata)
    model = KGCNN3.to(device)
    for i in range(5):
        model.load_state_dict(torch.load(dir_model+str(i)+".pth"))
        if i == 0:
            test_loader = DataLoader(
                dataset=fulldata,
                batch_size=1,
                sampler=torch.utils.data.SubsetRandomSampler(x[1])
            )
        elif i == 1:
            test_loader = DataLoader(
            dataset=fulldata,
            batch_size=1,
            sampler=torch.utils.data.SubsetRandomSampler(y[1])
            )
        elif i == 2:
            test_loader = DataLoader(
            dataset=fulldata,
            batch_size=1,
            sampler=torch.utils.data.SubsetRandomSampler(z[1])
            )
        elif i == 3:
            test_loader = DataLoader(
            dataset=fulldata,
            batch_size=1,
            sampler=torch.utils.data.SubsetRandomSampler(w[1])
            )
        elif i == 4:
            test_loader = DataLoader(
            dataset=fulldata,
            batch_size=1,
            sampler=torch.utils.data.SubsetRandomSampler(v[1])
            )
        model.eval()  # Set the model in evaluation mode

        Cd_outs = []

        Cd_ans = []

        test_ids = []
        with torch.no_grad():
            for id,images,A_char,Re,Cd in test_loader:

                id = id.to(device)
                images = Variable(images, requires_grad=True)
                A_char = Variable(A_char, requires_grad=True)
                Re = Variable(Re, requires_grad=True)
                Cd = Variable(Cd, requires_grad=True)

                A_char = A_char.view(A_char.size(0),1)
                Re = Re.view(Re.size(0),1)
                Cd = Cd.view(Cd.size(0),1)

                images = images.to(device)
                A_char = A_char.to(device)
                Re = Re.to(device)
                Cd = Cd.to(device)

                #with autograd.detect_anomaly():
                # Forward pass
                cd_out = model(images,Re)

                Cd_outs.append(torch.exp(cd_out).item())

                Cd_ans.append(torch.exp(Cd).item())

                test_ids.append(id.item())
                #err += (abs(outputs - t_Cd)/t_Cd)*100
                #print(err)
                #loss =  l + loss_mean
        #outs_act = np.array(outs)
        #answers_act = np.array(answers)
        Cd_outs = np.array(Cd_outs)
        Cd_acts = np.array(Cd_outs)

        Cd_ans = np.array(Cd_ans)
        Cd_actans = np.array(Cd_ans)

        test_ids = np.array(test_ids)
        mass_out = Cd_acts*0.5*cur_df['rho'][test_ids]*(cur_df['vel'][test_ids]**2)*cur_df['area'][test_ids]/9.81
        Cd_errs = ((Cd_ans - Cd_outs)/abs(Cd_ans))*100
        Cd_errs_abs = (abs(Cd_ans - Cd_outs)/abs(Cd_ans))*100
        Cd_aerrs = -((Cd_actans - Cd_acts)/abs(Cd_actans))*100
        Cd_aerrs_abs = (abs(Cd_actans - Cd_acts)/abs(Cd_actans))*100
        Cd_mse = (Cd_acts - Cd_actans)**2
        Cd_rmse = np.sqrt(np.sum(Cd_mse)/len(Cd_mse))
        Cd_nrmse = (Cd_rmse/np.mean(Cd_actans))*100
        mass_mse = (mass_out - cur_df['mass'][test_ids])**2
        mass_rmse = np.sqrt(np.sum(mass_mse)/len(mass_mse))
        predout = pd.DataFrame(Cd_outs,columns=['Cdout'])
        predout['massout'] = np.array(mass_out)
        predout['id'] = np.array(cur_df['id'][test_ids])
        predout['Cd'] = np.array(cur_df['Cd'][test_ids])
        predout['Re'] = np.array(cur_df['Re'][test_ids])
        predout['mass'] = np.array(cur_df['mass'][test_ids])
        hs_planout = pd.concat([hs_planout,predout],ignore_index=True)
# save outputs to csv file
hs_planout.to_csv(dir_savdat)